In [ ]:
# default_exp core
%load_ext autoreload
%autoreload 2

# Core

> Basic functions that could apply to many projects.

In [ ]:
#hide
from nbdev.showdoc import *

## Converting dictionaries to strings and visa versa.
> This is for saving experiment parameters easily.

In [ ]:
#export
def dict_to_str(d: dict, sep: str = '::') -> str:
    return sep.join(
        [str(k) + sep + str(v) for k, v in sorted(d.items())])

In [ ]:
dict_to_str(d={
    'name': 'f1',
    'foo': 3,
    'bar': 'zoo',
    'bool': True,
    'bool2': False,
    'float': 3.45})

'bar::zoo::bool::True::bool2::False::float::3.45::foo::3::name::f1'

In [ ]:
x = dict_to_str(d={
    'name': 'f1',
    'foo': 3,
    'bar': 'zoo',
    'bool': True,
    'bool2': False,
    'float': 3.45})

assert x == 'bar::zoo::'\
          + 'bool::True::'\
          + 'bool2::False::'\
          + 'float::3.45::'\
          + 'foo::3::'\
          + 'name::f1'

In [ ]:
#export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

In [ ]:
#export
class HelloSayer:
    "Say hello to `to` using `say_hello`"
    def __init__(self, to): self.to = to

    def say(self):
        "Do the saying"
        return say_hello(self.to)

In [ ]:
show_doc(HelloSayer)

<h2 id="HelloSayer" class="doc_header"><code>class</code> <code>HelloSayer</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>HelloSayer</code>(**`to`**)

Say hello to `to` using `say_hello`

In [ ]:
show_doc(HelloSayer.say)

<h4 id="HelloSayer.say" class="doc_header"><code>HelloSayer.say</code><a href="__main__.py#L6" class="source_link" style="float:right">[source]</a></h4>

> <code>HelloSayer.say</code>()

Do the saying

In [ ]:
hs = HelloSayer('Chris')
hs.say()

'Hello Chris!'

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
